# Лабораторная работа №3
Реализация Numba + Сравнение скорости

In [2]:
from typing import Tuple, Mapping

import numpy as np
import random
from matplotlib import pyplot as plt
import sklearn as skl
from sklearn import datasets
from IPython import display
import numba
from numba import jit
import IPython

# Функция Бута
$f(x,y) = (x + 2y - 7)^2 + (2x + y - 5)^2$

In [86]:
@numba.njit(fastmath=True, parallel=True)
def numba_GD(f: Mapping, df1: Mapping, df2: Mapping, x0: np.ndarray, y0: np.ndarray, lr: float = 0.01,
          iter: int = 100) -> np.ndarray:
    '''Моя простейшая реализация градиентного спуска.
    
    Args:
        f (Mapping): Функционал для оптимизации
        df (Mapping): Градиент оптимизирующего функционала
        x0 (np.ndarray): Стартовая точка 1
        y0 (np.ndarray): Стартовая точка 2
        lr (float): Скорость обучения. Default=0,01.
        iter (int): Количество итераций.
    
    Returns:
        Tuple [np.ndarray, np.ndarray, np.float32]: (x_optimal, f(x_optimal)).
    
    '''
    x_old = x0
    y_old = y0
    for i in range(iter):
        n1=df1(x0, y0)
        n2=df2(x0, y0)
        x_new = x_old - lr*n1
        y_new = y_old - lr*n2
    return [x_new, y_new, f(x_new, y_new)]

@numba.njit(fastmath=True)
def numba_Buta(x: np.ndarray, y: np.ndarray) -> np.float64:
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2

@numba.njit(fastmath=True)
def numba_grad_Buta_x(x: np.ndarray, y: np.ndarray) -> np.float64:
    return 10*x+8*y-34

@numba.njit(fastmath=True)
def numba_grad_Buta_y(x: np.ndarray, y: np.ndarray) -> np.float64:
    return 10*y + 8*x - 38

In [87]:
print(f'''f(1,3) = 0
Function: {numba_Buta(1,3)}
Gradient: {numba_grad_Buta_x(1,3)}, {numba_grad_Buta_y(1,3)}''')

f(1,3) = 0
Function: 0
Gradient: 0, 0


In [88]:
z, y = skl.datasets.make_blobs(n_samples=40000, centers=2, n_features=2, random_state=0)
y = np.zeros(40000)
x = np.zeros(40000)
for i in range(len(z)):
   x[i],y[i] = z[i]
print('X: ', x, '\nY: ', y)

X:  [ 1.58107779  2.02139889  1.74902357 ...  3.58479452 -0.5316157
  2.63024066] 
Y:  [-0.21815247  0.41577863  1.10999246 ...  0.55144109  4.041054
 -0.3872784 ]


In [89]:
result_GPU = numba_GD(numba_Buta, numba_grad_Buta_x, numba_grad_Buta_y, x, y)
print(f'''X: {result_GPU[0]}
Y: {result_GPU[1]}
Buta: {result_GPU[2]}''')

X: [ 1.78042221  2.12599671  1.82532181 ...  3.52219978 -0.46173845
  2.73819887]
Y: [ 0.05717655  0.59248886  1.23907133 ...  0.58951342  4.05947785
 -0.17896981]
Buta: [27.97318504 13.63309541  7.28346668 ... 12.22185665  3.90642696
 21.43046801]


# Замер времени с использованием Numba и без

In [90]:
%timeit -n100 numba_GD(numba_Buta, numba_grad_Buta_x, numba_grad_Buta_y, x, y)

14.4 ms ± 733 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [91]:
def GD(f: Mapping, df: Mapping, x0: np.ndarray, y0: np.ndarray, lr: float = 0.01,
          iter: int = 100) -> Tuple [np.ndarray, np.ndarray, np.float32]:
    '''Градиентный спуск.
    
    Args:
        f (Mapping): Функционал для оптимизации
        df (Mapping): Градиент оптимизирующего функционала
        x0 (np.ndarray): Стартовая точка 1
        y0 (np.ndarray): Стартовая точка 2
        lr (float): Скорость обучения. Default=0,01.
        iter (int): Количество итераций.
    
    Returns:
        Tuple [np.ndarray, np.ndarray, np.float32]: (x_optimal, f(x_optimal)).
        Кортеж с координатой x, координатой y и результатом функции
    
    '''
    x_old = x0
    y_old = y0
    for i in range(iter):
        n=df(x0, y0)
        x_new = x_old - lr*n[0]
        y_new = y_old - lr*n[1]
        
    return x_new, y_new, f(x_new, y_new)

def Buta(x: np.ndarray, y: np.ndarray) -> np.float64:
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2

def grad_Buta(x: np.ndarray, y: np.ndarray) -> np.float64:
    return np.array([10*x+8*y-34, 10*y + 8*x - 38])

In [92]:
%timeit -n100 GD(Buta, grad_Buta, x, y)

34.7 ms ± 3.15 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
